In [1]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [2]:
from s3ts.api.nets.encoders.frames.CNN_GAP import CNN_GAP_IMG

In [3]:
dm = load_tsdataset("WISDM", "./datasets/", 128, 8, 48, 1, True, None, [30, 31, 32, 33, 34, 35], True, 1)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Using 443636 observations for training and 154375 observations for validation and test


In [8]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "ts", "cnn_gap", "mlp", "cls", "test", 64, 64, 2, 0.001, voting={"n": 7, "rho": 0.1})

Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 128])
[tensor(128), 64, 64, 6]


In [9]:
train_model(dm, model, 3, {
    "default_root_dir": "training",
    "accelerator": "auto",
    "seed": 42
})

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type                      | Params
---------------------------------------------------------
0 | encoder    | CNN_GAP_TS                | 36.3 K
1 | decoder    | MultiLayerPerceptron      | 12.8 K
2 | flatten    | Flatten                   | 0     
3 | softmax    | Softmax                   | 0     
4 | train_cm   | MulticlassConfusionMatrix | 0     
5 | val_cm     | MulticlassConfusionMatrix | 0     
6 | val_auroc  | MulticlassAUROC           | 0     
7 | test_cm    | MulticlassConfusionMatrix | 0     
8 | test_auroc | MulticlassAUROC           | 0     
---------------------------------------------------------
49.1 K    Trainable params
0         Non-trainable params
49.1 K    Total params
0.196     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 3466/3466 [01:04<00:00, 53.48it/s, v_num=15, train_loss_step=0.00649, val_loss=0.686, val_auroc=0.971, val_re=0.813, train_loss_epoch=0.0491, train_re=0.480]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3466/3466 [01:04<00:00, 53.48it/s, v_num=15, train_loss_step=0.00649, val_loss=0.686, val_auroc=0.971, val_re=0.813, train_loss_epoch=0.0491, train_re=0.480]
Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 128])
[tensor(128), 64, 64, 6]
Validation DataLoader 0: 100%|██████████| 1207/1207 [00:11<00:00, 107.12it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_auroc         │    0.9737690091133118     │
│          val_f1           │    0.8315940499305725     │
│         val_loss          │    0.5829297304153442     │
│          val_pr           │    0.8384895920753479     │
│          val_re           │    0.8477136492729187     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (encoder): CNN_GAP_TS(
     (cnn_0): Sequential(
       (0): Conv1d(3, 32, kernel_size=(8,), stride=(1,), padding=same)
       (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
       (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_2): Sequential(
       (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
       (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
     )
   )
   (decoder): MultiLayerPerceptron(
     (fcn_layer_0): Linear(in_features=128, out_features=64, bias=True)
     (act_layer_0): ReL

In [7]:
import torchmetrics as tm

In [8]:
acc_online = tm.Accuracy(task="multiclass", num_classes=6, average="macro")
cm_online = tm.ConfusionMatrix(task="multiclass", num_classes=6)
dataloader = dm.val_dataloader()

In [9]:
model.eval()
for batch_idx, data in enumerate(dataloader):
    preds = model.logits(data["series"])
    acc_online(preds, data["label"])
    cl = torch.argmax(preds, dim=1)
    cm_online(cl, data["label"])


In [10]:
acc_online.compute()

tensor(0.8273)

In [11]:
cm = cm_online.compute()

In [12]:
TP = cm.diag()
FP = cm.sum(0) - TP
FN = cm.sum(1) - TP
TN = torch.empty(cm.shape[0])
for i in range(cm.shape[0]):
    TN[i] = cm[:i,:i].sum() + cm[:i,i:].sum() + cm[i:,:i].sum() + cm[i:,i:].sum()

In [13]:
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

In [14]:
accuracy

tensor([0.9405, 0.9863, 0.9907, 0.9917, 0.9052, 0.9293])

In [15]:
recall

tensor([0.6781, 0.9370, 0.8843, 0.8330, 0.8169, 0.8146])

In [16]:
recall.mean()

tensor(0.8273)